In [ ]:
import io
import requests
import pandas as pd
from datetime import date, timedelta

def get_df_from_json(data_url, record_path=['Data']):
    content = requests.get(data_url).json()
    df_all = pd.json_normalize(content,record_path=record_path)
    
    return df_all

# Start date and end date
today = date.today()
start_date = '2017-01-01'
end_date = date.today() - timedelta(days=1)

def collate_kaituna_data(start_date, end_date):
    # Get flow data
    flow_data_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=35946&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
    flow_df_all = get_df_from_json(flow_data_url)
    flow_df = flow_df_all[["TimeStamp", "Value"]]
    flow_df = flow_df.rename(columns={'Value': 'FlowRate'})
    flow_df['TimeStamp']= pd.to_datetime(flow_df['TimeStamp'])

    print(flow_df.shape)
    print(flow_df.head())
    
    # Get Lake levels
    lake_level_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=32419&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
    lake_level_df_all = get_df_from_json(lake_level_url)
    lake_level_df = lake_level_df_all[["TimeStamp", "Value"]]
    lake_level_df = lake_level_df.rename(columns={'Value': 'LakeLevel'})
    lake_level_df['TimeStamp']= pd.to_datetime(lake_level_df['TimeStamp'])
    print(lake_level_df.shape)
    print(lake_level_df.head())
    
    # Get gate levels
    #Gate 1
    gate_levels_url_1 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38970&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
    gate_1_df = get_df_from_json(gate_levels_url_1)
    gate_1_df = gate_1_df[["TimeStamp", "Value"]]
    gate_1_df = gate_1_df.rename(columns={"Value": "Gate1"})

    # Gate 2
    gate_levels_url_2 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38973&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
    gate_2_df = get_df_from_json(gate_levels_url_2)
    gate_2_df = gate_2_df[["TimeStamp", "Value"]]
    gate_2_df = gate_2_df.rename(columns={"Value": "Gate2"})

    # Gate 3
    gate_levels_url_3 = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=38972&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
    gate_3_df = get_df_from_json(gate_levels_url_3)
    gate_3_df = gate_3_df[["TimeStamp", "Value"]]
    gate_3_df = gate_3_df.rename(columns={"Value": "Gate3"})

    # Concatenate into single dataframe
    gate_levels_df_temp = pd.merge(gate_1_df, gate_2_df, on='TimeStamp')
    gate_levels_df = pd.merge(gate_levels_df_temp, gate_3_df, on='TimeStamp', how='left')
    gate_levels_df['TimeStamp']= pd.to_datetime(gate_levels_df['TimeStamp'])

    print(gate_levels_df.describe())
    print(gate_levels_df.head())

    # Get rainfall at Lake Rotoiti
    rainfall_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=47928&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)

    rainfall_df_all = get_df_from_json(rainfall_url)
    rainfall_df = rainfall_df_all[["TimeStamp", "Value"]]
    rainfall_df = rainfall_df.rename(columns={'Value': 'Rainfall'})
    rainfall_df['TimeStamp']= pd.to_datetime(rainfall_df['TimeStamp'])
    print(rainfall_df.head())
    print(rainfall_df.shape)
    
    # Merge datasets into one dataframe
    df_temp = pd.merge(lake_level_df, flow_df, on='TimeStamp', how='left')
    df_temp = pd.merge(df_temp, gate_levels_df, on='TimeStamp', how='left')
    df_combined = pd.merge(df_temp, rainfall_df, on='TimeStamp', how='right')

    #Set index
    df_combined['TimeStamp'] = pd.to_datetime(df_combined['TimeStamp'])
    df_combined.set_index('TimeStamp', inplace=True)

    #Remove rows for which we don't have rainfall data
    df_river_data = df_combined.dropna()

    #Sort by timestamp descending
    df_river_data = df_river_data.sort_values('TimeStamp')

    print (df_river_data)